In [6]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import random
import os
import imageio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from collections import Counter

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, RepeatedStratifiedKFold

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.applications import resnet
from keras.applications.resnet import ResNet50
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam

In [7]:
# If you are running from colab
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    print('Running on CoLab')
    directory = r'/content/drive/Shareddrives/cancer-detection-model/MERGED The IQ-OTHNCCD lung cancer dataset/Test Data'
else:
    # If you want to run local
    print('Not running on CoLab')
    directory = r'../../data/cancer-detection-model/CT-Scan Images for testing'

categories = ['Bengin cases', 'Malignant cases', 'Normal cases']

Not running on CoLab


In [8]:
# Set the random seed for reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [9]:
def get_test_generator(directory, image_height, image_width, batch_size):
    '''Build generators for train and validatio
        :directory: Location of the data
        :image_height: image height
        :image_width: image width
        :batch_size: The batch size
    '''

    datagen = ImageDataGenerator(dtype='float32',
        rescale=1.0 / 255.0)

    test_generator = datagen.flow_from_directory(
        directory,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical',  # 'categorical' for multi-class classification
        shuffle=False,
        classes={'Benign': 0, 
                 'Malignant': 1, 
                 'Normal': 2}
    )

    return test_generator

In [10]:
models = {'lung_cancer_detection_One.RESNET50V2': 
          {'image_height': 224, 'image_width': 224, 'batch_size': 10},
        'lung_cancer_detection_One.SMOTE': 
          {'image_height': 256, 'image_width': 256, 'batch_size': 8}}


for model, value in models.items():
    # To load the model architecture and weights in a new session:
    loaded_model = tf.keras.models.load_model('lung_cancer_detection_One.RESNET50V2')

    # Compile the loaded model
    loaded_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    test_generator = get_test_generator(directory, value['image_width'], value['image_height'], value['batch_size'])
    test_loss, test_accuracy = loaded_model.evaluate(test_generator)
    print(f"{model} Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Found 364 images belonging to 3 classes.
37/37 [==============================] - 13s 317ms/step - loss: 2.6652 - accuracy: 0.6044
lung_cancer_detection_One.RESNET50V2 Test Loss: 2.6651716232299805, Test Accuracy: 0.6043956279754639
Found 364 images belonging to 3 classes.
46/46 [==============================] - 18s 369ms/step - loss: 2.2174 - accuracy: 0.6016
lung_cancer_detection_One.SMOTE Test Loss: 2.217400312423706, Test Accuracy: 0.6016483306884766
